## <center>Error for </center>

\begin{cases}
y'(t)= ay(t-\tau),&\\
y(t)=1 for all t<0&
\end{cases}

## <center>with IE </center>

(Take a=\pi / 2)

In [1]:
from math import factorial

## Right-hand-side of the simple DDE
def simple(a):
    def simple_dde(t, y1, y2):
        return -a*y2
    return simple_dde


## Constant one history function
def phi_0(t):
    return 1


def simple_exact(a, t_0, t_f, tau, N): 
    if t_0 != 0:
        raise ValueError("t_0 should be set to 0")
    t = np.linspace(t_0, t_f, N+1)
    y = np.zeros(len(t))
    for i, ti in enumerate(t):
        n = int(np.floor(ti / tau)) + 1 
        y[i] = 1
        for k in range(1, n+1):
            y[i] += (-a)**k * (ti - (k-1)*tau)**k / factorial(k)
    return y

In [2]:
import numpy as np

## Linear interpolation for delay terms
def y_delayed(t, y, t_q):
    j = np.searchsorted(t, t_q)  #Busca el índice donde insertar t_q para mantener el arreglo ordenado.
    j1, j2 = j-1, j     #Toma los dos puntos de la malla que rodean a t_q
    w = (t_q - t[j1]) / (t[j2] - t[j1])      #Aplica interpolación en estas 2 líneas
    y_delayed = (1 - w) * y[j1] + w * y[j2]
    return y_delayed

from scipy.optimize import fsolve

## Implicit Euler
def IE(f, phi, t_0, t_f, tau, N, history=False):

    ## Preliminaries
    h = (t_f - t_0) / N
    t = np.linspace(t_0, t_f, N+1)
    y = np.zeros(len(t))
    y[0] = phi(t_0)
    
    ## IE with linear interpolation for delay term
    for i in range(0, N):
        if t[i] < tau:
            y_impl = lambda x : -x + y[i] + h*f(t[i+1], x, phi(t[i+1] - tau))
            y[i+1] = fsolve(y_impl, [y[i]])[0]
        else:
            y_impl = lambda x : -x + y[i] + h*f(t[i+1], x, y_delayed(t, y, t[i+1] - tau))
            y[i+1] = fsolve(y_impl, [y[i]])[0]

    if history == True: ## Wether the solution should include the history (values on the interval [t_0 - tau, t_0])
        t_hist = np.linspace(t_0-tau, t_0, int(N / (t_f // tau) + 1))
        y_hist = np.array([phi(ti) for ti in t_hist])
        t = np.concatenate((t_hist, t))
        y = np.concatenate((y_hist, y))

    return h, t, y

In [10]:
#Initial values
a = np.pi / 2
t_0 = 0
t_f = 30
tau = 1


maxnormresult = []
orderresult = []

N = 45 #I changed N (again)


    
for i in range(3,10):
    h, t, y = IE(simple(a), phi_0, t_0, t_f, tau, N*2**i, False)
    exact = simple_exact(a, t_0, t_f, tau, N*2**i)
    maxnorm = np.linalg.norm(exact - y,np.inf)
    maxnormresult.append(maxnorm)
    
errorvector = np.hstack(maxnormresult)  


for i in range(0,len(errorvector)-1):
    order = np.log2(errorvector[i]/errorvector[i+1])
    orderresult.append(order)
    
print(orderresult)
print(errorvector)

C:\Users\natal\AppData\Local\Temp\ipykernel_30172\1367267053.py:26: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  y[i+1] = fsolve(y_impl, [y[i]])[0]
C:\Users\natal\AppData\Local\Temp\ipykernel_30172\1367267053.py:29: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  y[i+1] = fsolve(y_impl, [y[i]])[0]


[np.float64(0.6363500406115682), np.float64(0.874008526390301), np.float64(0.9573687217427712), np.float64(0.9634014987831582), np.float64(0.9816386422422142), np.float64(0.9908736081317793)]
[1.09094578 0.70184744 0.38294811 0.19721649 0.10114175 0.05121861
 0.02577182]
